In [1]:
import collections
import pandas as pd
import datetime
import numpy as np
import json
import codecs

In [2]:
data_fr = pd.read_csv('../data/data_fr_vf.csv', sep=';', usecols=np.arange(2,38))
corres_ICD10 = pd.read_csv('../data/corres_ICD10.csv', sep=';',usecols=[1,2,3,4,5])
corres_geo = pd.read_csv('../data/corres_geo.csv', sep=';', usecols=[1,2,3,4])

In [3]:
# Nettoyage des données:
for col in data_fr.columns[15:]:
    data_fr[col] = data_fr[col].str.strip().replace({":":"NaN", ": z":"NaN"}).astype('float64')

In [4]:
data_fr.head()

,sex,age,geo_niv_0,geo_niv_1,geo_niv_2,region,icd10_niv_0,icd10_nom_niv_0,icd10_niv_1,icd10_nom_niv_1,...,2003,2002,2001,2000,1999,1998,1997,1996,1995,1994
0,F,TOTAL,FR,FR1,FR10,ile de France,A-R_V-Y,Toutes causes de mortalite,A-R_V-Y,Toutes causes de mortalite,...,684.3,640.7,649.4,649.2,660.7,670.5,671.1,681.2,686.3,669.9
1,F,TOTAL,FR,FR2,FR21,Champagne-Ardenne,A-R_V-Y,Toutes causes de mortalite,A-R_V-Y,Toutes causes de mortalite,...,898.5,866.6,870.9,877.6,900.1,858.5,861.1,874.0,881.0,842.5
2,F,TOTAL,FR,FR2,FR22,Picardie,A-R_V-Y,Toutes causes de mortalite,A-R_V-Y,Toutes causes de mortalite,...,880.7,844.1,830.9,829.0,839.5,828.4,838.8,844.8,834.7,794.2
3,F,TOTAL,FR,FR2,FR23,Haute-Normandie,A-R_V-Y,Toutes causes de mortalite,A-R_V-Y,Toutes causes de mortalite,...,867.2,827.0,833.0,820.2,814.5,830.9,811.7,844.8,833.0,777.1
4,F,TOTAL,FR,FR2,FR24,Centre,A-R_V-Y,Toutes causes de mortalite,A-R_V-Y,Toutes causes de mortalite,...,1004.7,941.5,934.8,934.6,949.7,933.6,928.5,928.5,932.6,894.4


In [5]:
data_fr.to_csv('data_fr.csv')

In [9]:
# data_fr.drop(['geo_niv_2', 'icd10_niv_3'], inplace=True, axis=1)

In [37]:
# df = data_fr.pivot_table(index=data_fr.columns[2:9].to_list(),
#                    values=data_fr.columns[9:].to_list(),
#                    columns=data_fr.columns[:2].to_list())
# # df.to_json('arbo_1.json')

# df

# df = data_fr.pivot_table(index=data_fr.columns[2:9].to_list(),
#                    values=data_fr.columns[9:].to_list(),
#                    columns=data_fr.columns[:1].to_list())
# # df.to_json('arbo_2.json')

# df.head()

In [58]:
# ============ Dictionnaires imbriqués ===========
# ================================================

# def nested_dict():
#     return collections.defaultdict(nested_dict)
# result = nested_dict()

# for row in df.itertuples():
#     result[row.Index[0]][row.Index[1]][row.Index[2]][row.Index[3]][row.Index[4]][row.Index[5]] = row._1
    
# out = json.dumps(result)

# with open('test.json', 'w') as f:
#     f.write(out)